# Gerador de Tweets utilizando LSTM

## Parte 1 - Extraindo Tweets com GetOldTweets3
Nesta seção, vamos:
 * Extrair tweets, dado nome do usuário e a quantidade de tweets a ser extraido
 * Salvar os dados
 
PS: Em 18/09/2020, o Twitter alterou a saída do endpoint /i/search/timeline, impossibilitando a extração com o GetOldTweets3 após esta data (issue: https://github.com/Mottl/GetOldTweets3/issues/98). Para reproduzir a extração, recomenda-se utilizar o Twitter API, limitado à extração de 5000 tweets por mês.
Na próxima seção, vamos treinar um modelo, utilizando como exemplo os tweets do autor.

In [2]:
#Parte 1
import GetOldTweets3 as got
import pandas as pd

#Parte 2

import spacy
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

import random
random.seed(101)

In [ ]:
#Extração dos tweets

def username_tweets_to_csv(username, count):
    tweetCriteria = got.manager.TweetCriteria().setUsername(username).setMaxTweets(count)
    tweets = got.manager.TweetManager.getTweets(tweetCriteria)
    tweets_list = [[tweet.date, tweet.text] for tweet in tweets]
    df = pd.DataFrame(tweets_list, columns = ['Datetime', 'Text'])    
    return df

In [ ]:
#Execução da extração

username = 'user_name'
count = 3000

df = username_tweets_to_csv(username, count)


In [ ]:
#Conversão dos dados para csv

df.to_csv(f'{username}-{int(count/1000)}k-tweets.csv', sep=',')

## Parte 2 - Pré-processamento dos dados

Objetivos desta seção:
 * Tratamento e limpeza do texto
 * Criação de séries sequenciais
 * Tokenização dos dados

In [3]:
#Parâmetros:

#Tamanho das sequencias
seq_len = 20

In [4]:
df = pd.read_csv('fastererick-3k-tweets.csv')

In [5]:
df.head(10)

,Datetime,Text
0,2020-08-17 01:04:55+00:00,4 Elites do boy e 3 meus hehe #SmashBros #Nint...
1,2020-06-27 01:26:56+00:00,#AnimalCrossing #ACNH #NintendoSwitch
2,2020-06-19 17:37:42+00:00,#AnimalCrossing #ACNH #NintendoSwitch
3,2020-01-14 21:22:15+00:00,Pra pensar na bad: O que seu personagem favori...
4,2020-01-14 21:08:00+00:00,Medo de largar tudo
5,2020-01-10 14:28:39+00:00,Ainda qro comprar munchkin aaaaaaa
6,2019-12-27 06:25:03+00:00,Perdi
7,2019-12-26 02:06:54+00:00,Paçoca
8,2019-12-25 05:55:31+00:00,Queria tar o 2 contigo hj
9,2019-12-24 23:55:27+00:00,"Psé, Facebook ta uma bosta, mano hahaha'"


In [6]:
df = df.loc[:,'Text']
df

0       4 Elites do boy e 3 meus hehe #SmashBros #Nint...
1                   #AnimalCrossing #ACNH #NintendoSwitch
2                   #AnimalCrossing #ACNH #NintendoSwitch
3       Pra pensar na bad: O que seu personagem favori...
4                                     Medo de largar tudo
                              ...                        
2454    Curioso como o twitter, o orkut e o forms está...
2455    @caasousaa disponha ^^ Ainda sou novo no twitt...
2456                                     Tarja Turunen S2
2457    Domingo tedioso... e podia estar assistindo o ...
2458    Yeah fiz outro Twitter... Aquele outro já tava...
Name: Text, Length: 2459, dtype: object

In [7]:
# Configurações para tratamento do texto

nlp = spacy.load('en_core_web_sm', disable=['parser', 'tagger', 'ner'])
nlp.max_length = 2000000

In [8]:
# Configuração individual de cada tweet
def customize_text(df):
    df = df.str.replace(r'([@#][\w_-]+)',' ')
    df = df.str.lstrip().str.rstrip()
    return df

df = customize_text(df)
df

0                           4 Elites do boy e 3 meus hehe
1                                                        
2                                                        
3       Pra pensar na bad: O que seu personagem favori...
4                                     Medo de largar tudo
                              ...                        
2454    Curioso como o twitter, o orkut e o forms está...
2455    disponha ^^ Ainda sou novo no twitter pegando ...
2456                                     Tarja Turunen S2
2457    Domingo tedioso... e podia estar assistindo o ...
2458    Yeah fiz outro Twitter... Aquele outro já tava...
Name: Text, Length: 2459, dtype: object

In [9]:
# União dos tweets em uma única string
def join_text(df):
    df = df.values
    df_string = ' '.join(df)
    return df_string

df_string = join_text(df)
df_string[0:500]

"4 Elites do boy e 3 meus hehe   Pra pensar na bad: O que seu personagem favorito de filme/jogo diria? Medo de largar tudo Ainda qro comprar munchkin aaaaaaa Perdi Paçoca Queria tar o 2 contigo hj Psé, Facebook ta uma bosta, mano hahaha' Nem pra minha mãe ter esperado pra bater na minha tia crente hj e não no mês passado, aí n tava esse tédio todo aq Tedio ta tao grande aq q ate voltei É aqui o muro das lamentações socialmente aceito? Topo N consegui votar em vc :( Oi Não pode ( ) A ( ) B (X) C o"

In [10]:
# Limpeza das pontuações indesejadas e geração de tokens
def separate_punc(doc_text):
    doc_text = doc_text.replace('...','')
    return [token.text.lower() for token in nlp(doc_text) if token.text not in "\n\n \n\n\n!'\"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n "]

tokens = separate_punc(df_string)
tokens[0:10]

['4', 'elites', 'do', 'boy', 'e', '3', 'meus', 'hehe', '  ', 'pra']

In [11]:
#Geração de tokens sequenciais

train_len = seq_len + 1
text_sequences = []
for i in range(train_len, len(tokens)):
    seq = tokens[i-train_len:i]    
    text_sequences.append(seq)

text_sequences[0]

['4',
 'elites',
 'do',
 'boy',
 'e',
 '3',
 'meus',
 'hehe',
 '  ',
 'pra',
 'pensar',
 'na',
 'bad',
 'o',
 'que',
 'seu',
 'personagem',
 'favorito',
 'de',
 'filme',
 'jogo']

In [12]:
#Conversão de strings para labels numéricos

tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences)
sequences[0]

[341,
 4879,
 7,
 4878,
 5,
 246,
 245,
 4877,
 6,
 9,
 678,
 15,
 4876,
 3,
 11,
 124,
 4874,
 1294,
 1,
 1988,
 458]

In [13]:
#Tamanho do vocabulário
vocabulary_size = len(tokenizer.word_counts)
vocabulary_size

4880

In [14]:
#Conversão para array
sequences = np.array(sequences)
sequences[0]

array([ 341, 4879,    7, 4878,    5,  246,  245, 4877,    6,    9,  678,
         15, 4876,    3,   11,  124, 4874, 1294,    1, 1988,  458])

In [15]:
#Criação das variaveis de treino
X = sequences[:,:-1]
y = sequences[:,-1]

In [16]:
X[0]

array([ 341, 4879,    7, 4878,    5,  246,  245, 4877,    6,    9,  678,
         15, 4876,    3,   11,  124, 4874, 1294,    1, 1988])

In [17]:
y[0]

458

In [18]:
# Categorização do y
y = to_categorical(y,num_classes=vocabulary_size+1)
y[0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

## Parte 3 - Criação do modelo LSTM
Nesta parte, vamos:
 * gerar o modelo LSTM
 * treinar o modelo

In [19]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from pickle import dump

In [20]:
#Parâmetros
model_name = 'fastererick_twitter'

In [21]:
#Criação do modelo

def create_model(vocabulary_size, seq_len):  
    model = Sequential()
    model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
    model.add(LSTM(150, return_sequences=True))
    model.add(LSTM(150))
    model.add(Dense(50,activation='relu'))
    model.add(Dense(vocabulary_size,activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])    
    return model

model = create_model(vocabulary_size+1, seq_len)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 20)            97620     
_________________________________________________________________
lstm (LSTM)                  (None, 20, 150)           102600    
_________________________________________________________________
lstm_1 (LSTM)                (None, 150)               180600    
_________________________________________________________________
dense (Dense)                (None, 50)                7550      
_________________________________________________________________
dense_1 (Dense)              (None, 4881)              248931    
Total params: 637,301
Trainable params: 637,301
Non-trainable params: 0
_________________________________________________________________


In [22]:
#Treinamento do modelo
history = model.fit(X, y, batch_size=128, epochs=600, verbose=1)

Epoch 1/600
204/204 [==============================] - 15s 75ms/step - loss: 7.2932 - accuracy: 0.0203
Epoch 2/600
204/204 [==============================] - 16s 79ms/step - loss: 6.9509 - accuracy: 0.0215
Epoch 3/600
204/204 [==============================] - 17s 81ms/step - loss: 6.9326 - accuracy: 0.0225
Epoch 4/600
204/204 [==============================] - 17s 83ms/step - loss: 6.8465 - accuracy: 0.0213
Epoch 5/600
204/204 [==============================] - 16s 79ms/step - loss: 6.7147 - accuracy: 0.0295
Epoch 6/600
204/204 [==============================] - 16s 76ms/step - loss: 6.5930 - accuracy: 0.0339
Epoch 7/600
204/204 [==============================] - 16s 78ms/step - loss: 6.5141 - accuracy: 0.0353
Epoch 8/600
204/204 [==============================] - 16s 78ms/step - loss: 6.5100 - accuracy: 0.0341
Epoch 9/600
204/204 [==============================] - 16s 79ms/step - loss: 6.4591 - accuracy: 0.0386
Epoch 10/600
204/204 [==============================] - 16s 79ms/step - l

204/204 [==============================] - 16s 79ms/step - loss: 2.9270 - accuracy: 0.3556
Epoch 80/600
204/204 [==============================] - 16s 78ms/step - loss: 2.8948 - accuracy: 0.3596
Epoch 81/600
204/204 [==============================] - 16s 81ms/step - loss: 2.8580 - accuracy: 0.3658
Epoch 82/600
204/204 [==============================] - 16s 78ms/step - loss: 2.8324 - accuracy: 0.3739
Epoch 83/600
204/204 [==============================] - 16s 81ms/step - loss: 2.8022 - accuracy: 0.3785
Epoch 84/600
204/204 [==============================] - 16s 79ms/step - loss: 2.7832 - accuracy: 0.3788
Epoch 85/600
204/204 [==============================] - 16s 79ms/step - loss: 2.7503 - accuracy: 0.3871
Epoch 86/600
204/204 [==============================] - 16s 78ms/step - loss: 2.7124 - accuracy: 0.3964
Epoch 87/600
204/204 [==============================] - 17s 82ms/step - loss: 2.6787 - accuracy: 0.4014
Epoch 88/600
204/204 [==============================] - 16s 78ms/step - loss:

204/204 [==============================] - 16s 79ms/step - loss: 1.2806 - accuracy: 0.6918
Epoch 157/600
204/204 [==============================] - 17s 81ms/step - loss: 1.2702 - accuracy: 0.6929
Epoch 158/600
204/204 [==============================] - 16s 80ms/step - loss: 1.2710 - accuracy: 0.6903
Epoch 159/600
204/204 [==============================] - 17s 81ms/step - loss: 1.2411 - accuracy: 0.6977
Epoch 160/600
204/204 [==============================] - 16s 80ms/step - loss: 1.2341 - accuracy: 0.7022
Epoch 161/600
204/204 [==============================] - 16s 79ms/step - loss: 1.2448 - accuracy: 0.6956
Epoch 162/600
204/204 [==============================] - 16s 79ms/step - loss: 1.2552 - accuracy: 0.6934
Epoch 163/600
204/204 [==============================] - 16s 79ms/step - loss: 1.2323 - accuracy: 0.6977
Epoch 164/600
204/204 [==============================] - 16s 80ms/step - loss: 1.1944 - accuracy: 0.7076
Epoch 165/600
204/204 [==============================] - 16s 79ms/ste

204/204 [==============================] - 18s 88ms/step - loss: 0.7194 - accuracy: 0.8082
Epoch 233/600
204/204 [==============================] - 17s 83ms/step - loss: 0.6041 - accuracy: 0.8421
Epoch 234/600
204/204 [==============================] - 18s 87ms/step - loss: 0.5263 - accuracy: 0.8717
Epoch 235/600
204/204 [==============================] - 18s 86ms/step - loss: 0.4541 - accuracy: 0.8962
Epoch 236/600
204/204 [==============================] - 19s 91ms/step - loss: 0.4209 - accuracy: 0.9057
Epoch 237/600
204/204 [==============================] - 18s 87ms/step - loss: 0.3869 - accuracy: 0.9154
Epoch 238/600
204/204 [==============================] - 18s 89ms/step - loss: 0.4058 - accuracy: 0.9092
Epoch 239/600
204/204 [==============================] - 19s 93ms/step - loss: 0.5262 - accuracy: 0.8681
Epoch 240/600
204/204 [==============================] - 19s 95ms/step - loss: 0.7061 - accuracy: 0.8091
Epoch 241/600
204/204 [==============================] - 19s 95ms/ste

204/204 [==============================] - 17s 83ms/step - loss: 0.6059 - accuracy: 0.8323
Epoch 310/600
204/204 [==============================] - 17s 84ms/step - loss: 0.3610 - accuracy: 0.9044
Epoch 311/600
204/204 [==============================] - 17s 84ms/step - loss: 0.2458 - accuracy: 0.9454
Epoch 312/600
204/204 [==============================] - 17s 83ms/step - loss: 0.1779 - accuracy: 0.9670
Epoch 313/600
204/204 [==============================] - 17s 84ms/step - loss: 0.1355 - accuracy: 0.9794
Epoch 314/600
204/204 [==============================] - 17s 84ms/step - loss: 0.1174 - accuracy: 0.9839
Epoch 315/600
204/204 [==============================] - 17s 84ms/step - loss: 0.1153 - accuracy: 0.9856
Epoch 316/600
204/204 [==============================] - 17s 83ms/step - loss: 0.1154 - accuracy: 0.9844
Epoch 317/600
204/204 [==============================] - 17s 84ms/step - loss: 0.1096 - accuracy: 0.9857
Epoch 318/600
204/204 [==============================] - 17s 82ms/ste

204/204 [==============================] - 20s 97ms/step - loss: 0.0991 - accuracy: 0.9845
Epoch 387/600
204/204 [==============================] - 20s 97ms/step - loss: 0.2146 - accuracy: 0.9452
Epoch 388/600
204/204 [==============================] - 20s 97ms/step - loss: 0.6376 - accuracy: 0.8200
Epoch 389/600
204/204 [==============================] - 20s 96ms/step - loss: 0.6094 - accuracy: 0.8294
Epoch 390/600
204/204 [==============================] - 19s 94ms/step - loss: 0.3607 - accuracy: 0.8967
Epoch 391/600
204/204 [==============================] - 20s 97ms/step - loss: 0.1928 - accuracy: 0.9503
Epoch 392/600
204/204 [==============================] - 19s 95ms/step - loss: 0.1178 - accuracy: 0.9791
Epoch 393/600
204/204 [==============================] - 19s 94ms/step - loss: 0.0838 - accuracy: 0.9879
Epoch 394/600
204/204 [==============================] - 19s 94ms/step - loss: 0.0659 - accuracy: 0.9926
Epoch 395/600
204/204 [==============================] - 20s 96ms/ste

204/204 [==============================] - 20s 100ms/step - loss: 0.0226 - accuracy: 0.9994
Epoch 463/600
204/204 [==============================] - 20s 100ms/step - loss: 0.0189 - accuracy: 0.9996
Epoch 464/600
204/204 [==============================] - 20s 96ms/step - loss: 0.0303 - accuracy: 0.9979
Epoch 465/600
204/204 [==============================] - 19s 91ms/step - loss: 0.2887 - accuracy: 0.9266
Epoch 466/600
204/204 [==============================] - 20s 98ms/step - loss: 1.0840 - accuracy: 0.7336
Epoch 467/600
204/204 [==============================] - 20s 96ms/step - loss: 0.5141 - accuracy: 0.8555
Epoch 468/600
204/204 [==============================] - 20s 97ms/step - loss: 0.2554 - accuracy: 0.9283
Epoch 469/600
204/204 [==============================] - 21s 103ms/step - loss: 0.1422 - accuracy: 0.9651
Epoch 470/600
204/204 [==============================] - 21s 102ms/step - loss: 0.0807 - accuracy: 0.9871
Epoch 471/600
204/204 [==============================] - 22s 106m

204/204 [==============================] - 20s 99ms/step - loss: 0.5185 - accuracy: 0.8572
Epoch 540/600
204/204 [==============================] - 22s 108ms/step - loss: 0.2070 - accuracy: 0.9426
Epoch 541/600
204/204 [==============================] - 21s 101ms/step - loss: 0.1207 - accuracy: 0.9752
Epoch 542/600
204/204 [==============================] - 22s 108ms/step - loss: 0.0583 - accuracy: 0.9922
Epoch 543/600
204/204 [==============================] - 19s 95ms/step - loss: 0.0673 - accuracy: 0.9905
Epoch 544/600
204/204 [==============================] - 19s 92ms/step - loss: 0.0369 - accuracy: 0.9974
Epoch 545/600
204/204 [==============================] - 20s 99ms/step - loss: 0.0252 - accuracy: 0.9990
Epoch 546/600
204/204 [==============================] - 21s 101ms/step - loss: 0.0211 - accuracy: 0.9994
Epoch 547/600
204/204 [==============================] - 20s 98ms/step - loss: 0.0157 - accuracy: 0.9998
Epoch 548/600
204/204 [==============================] - 20s 98ms

In [23]:
#Serialização do modelo e do tokenizer
model.save(f'{model_name}.h5')
dump(tokenizer,open(f'{model_name}.p', 'wb'))

## Parte 4 - Testando o modelo
Finalmente, vamos:
 * Carregar o modelo
 * Gerar textos a partir do modelo treinado

In [24]:
from pickle import load
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model

In [25]:
#Parâmetros
model_name = 'fastererick_twitter'


In [26]:
#Carregamento do modelo

model = load_model(f'{model_name}.h5')
tokenizer = load(open(f'{model_name}.p', 'rb'))

In [27]:
#Função para gerar o texto

def generate_text(model,tokenizer, seq_len, seed, num_words):
    
    output_text = []
    
    input_text = seed
    
    for i in range(num_words):
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre') #Padding caso haja menos seq_len do que o total, trunca no inicio da string
        pred_word_index = model.predict_classes(pad_encoded, verbose=0)[0]
        pred_word = tokenizer.index_word[pred_word_index]
        input_text += ' '+pred_word
        
        output_text.append(pred_word)
        
    output_text = ' '.join(output_text)
    return output_text

In [28]:
#Geração do seed
def generate_seed():
    random_pick = random.randint(0, len(text_sequences))
    random_seed = text_sequences[random_pick]
    seed = ' '.join(random_seed)
    return seed

In [30]:
#Geração do texto

num_words = 25
for _ in range(100):
    seed = generate_seed()
    text = generate_text(model,tokenizer,seq_len,seed=seed, num_words=num_words)
    print(f"ErickBot: {text}\n")

ErickBot: primo experimente casal bonito antes kkkkkk eu ja 6 abraço nova maldita poética .- .- .- .- .- .- .- .- .- .- .- .-

ErickBot: add ^^ ^^ ^^ ^^ ^^ ^^ ^^ ^^ ^^ ^^ ^^ ^^ ^^ ^^ ^^ ^^ ^^ ^^ ^^ ^^ ^^ ^^ ^^ ^^

ErickBot: e metade burra omg ahuahuahuhua kkkkkkkkkkkkkkkkk neh xd nao soh t seguira no twitter como ira ajudar a proliferar o avatar munitinho ahuahuahua kkkkkkkkkkk nada

ErickBot: kk eh melhor por uma leituras gêmeo ironicamente kkkkkkkkkkkkkkkk na péssima deixou o chefao pro inves de 2012 achei um novo pra tirar um livro

ErickBot: bronca sua tcc .- .- .- .- .- .- .- .- .- .- .- .- .- .- .- .- .- .- .- .- .- .-

ErickBot: seu subita fim de tarde amanha animal provei q foi s n e boa coisa mexendo mais isso vai gastar boas coragem a menos ou

ErickBot: estão um cartas confortavel e a crowling geo eh de oi depois em quali vem ok mew amor vou ir hj ta foda agora como

ErickBot: se as notas do técnico já saíram .- .- .- .- .- .- .- .- .- .- .- .- .- .- .- .- .- .-

ErickBot: d

ErickBot: .- .- .- .- .- .- .- .- .- .- .- .- .- .- .- .- .- .- .- .- .- .- .- .- .-

ErickBot: =) =) =) =) =) =) =) =) =) =) =) =) =) =) =) =) =) =) =) =) =) =) =) =) =)

ErickBot: rockeiro deve ter para ele vdd não eh me pessoa um do cel nos 25 o                           

ErickBot: haha me livrei de uma encrenca antes mesmo que ela acontecesse \o/ banho haha msg ok kkkkkkk o seu bom de calor chato agora cabulei

ErickBot: msm xd i de nota etec a nota eh por letra p depois do banho a raiva passou =) =) =) =) =) =) =)

ErickBot: q to usando certeza mario isso estudar tira e boi =) =) =) =) =) =) =) =) =) =) =) =) =) =) =)

ErickBot: sim soh q me fez muito doendo o.o satelite haha faz 20 com as som pra sexta mais ser nessa › q tava aceitar ja

ErickBot: 31".- 31".- 31".- 31".- 31".- 31".- 31".- 31".- 31".- 31".- 31".- 31".- 31".- 31".- 31".- 31".- 31".- 31".- 31".- 31".- 31".- 31".- 31".- 31".- 31".-

ErickBot: de comer bolo mas ta valendo ow preciso falar urgente c vc leticia alias o